In [153]:
import pandas as pd

In [154]:

# Load CSV
df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/BenignTraffic.csv')


df['Label'] = 'BenignTraffic' 



# Save updated CSV
df.to_csv('labeled_dataset_benign.csv', index=False)
print(df)

        stream                       src_mac             dst_mac  \
0            0          Arlo Q Indoor Camera   3c:18:a0:41:c3:a0   
1            0          Arlo Q Indoor Camera   3c:18:a0:41:c3:a0   
2            0          Arlo Q Indoor Camera   3c:18:a0:41:c3:a0   
3            0          Arlo Q Indoor Camera   3c:18:a0:41:c3:a0   
4            0          Arlo Q Indoor Camera   3c:18:a0:41:c3:a0   
...        ...                           ...                 ...   
297058       0  HeimVision Smart WiFi Camera   01:00:5e:7f:ff:fa   
297059    7998             3c:18:a0:41:c3:a0   ac:17:02:05:34:27   
297060    7998             3c:18:a0:41:c3:a0   ac:17:02:05:34:27   
297061   15507               Home Eye Camera   3c:18:a0:41:c3:a0   
297062       1             3c:18:a0:41:c3:a0  Nest Indoor Camera   

                 src_ip           dst_ip  src_port  dst_port  \
0       192.168.137.175     99.81.244.93     56891       443   
1       192.168.137.175     99.81.244.93     56891     

In [155]:
# Load CSV
df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/Spoofing/DNS_Spoofing.csv')

df['Label'] = 'DNS_Spoofing' 



# Save updated CSV
df.to_csv('labeled_dataset_DNS_spoofing.csv', index=False)
print(df)

C:\Users\nazih\AppData\Local\Temp\ipykernel_15908\205077514.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/Spoofing/DNS_Spoofing.csv')


        stream                  src_mac                       dst_mac  \
0           -1  Netatmo Weather Station             ff:ff:ff:ff:ff:ff   
1           -1              Wyze Camera             ff:ff:ff:ff:ff:ff   
2           -1              Wyze Camera             e8:1b:69:f8:d4:e3   
3           -1              Wyze Camera             ff:ff:ff:ff:ff:ff   
4           -1        3c:18:a0:41:c3:a0       Netatmo Weather Station   
...        ...                      ...                           ...   
296020      35        3c:18:a0:41:c3:a0  HeimVision Smart WiFi Camera   
296021      -1        3c:18:a0:41:c3:a0                Netatmo Camera   
296022    4061              Wyze Camera             3c:18:a0:41:c3:a0   
296023      -1  Netatmo Weather Station             ff:ff:ff:ff:ff:ff   
296024    6764        3c:18:a0:41:c3:a0                   Wyze Camera   

                src_ip           dst_ip  src_port  dst_port  \
0                  NaN              NaN         0         0 

In [156]:
# Load CSV
df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/Spoofing/MITM-ArpSpoofing.csv')

# Add a label column (manually assign for small datasets)
df['Label'] = 'MITM-ArpSpoofing'

# Save updated CSV
df.to_csv('labeled_dataset_MITM_ArpSpoofing.csv', index=False)
print(df)

C:\Users\nazih\AppData\Local\Temp\ipykernel_15908\1053915760.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/Spoofing/MITM-ArpSpoofing.csv')


        stream             src_mac            dst_mac           src_ip  \
0            0         Wyze Camera  3c:18:a0:41:c3:a0   192.168.137.11   
1            0         Wyze Camera  3c:18:a0:41:c3:a0   192.168.137.11   
2            0         Wyze Camera  3c:18:a0:41:c3:a0   192.168.137.11   
3            0         Wyze Camera  3c:18:a0:41:c3:a0   192.168.137.11   
4            1      Netatmo Camera  3c:18:a0:41:c3:a0   192.168.137.48   
...        ...                 ...                ...              ...   
290432       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   
290433       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   
290434       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   
290435       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   
290436       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   

                 dst_ip  src_port  dst_port  inter_arrival_time  \
0           35.86.13.17     39807       443 

In [157]:
import pandas as pd

# Load datasets into a dictionary
datasets = {
    'benign': pd.read_csv('labeled_dataset_benign.csv'),
    'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
    'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')
}

# Verify loaded data
for name, df in datasets.items():
    print(f"{name} dataset shape: {df.shape}")
    print(df.head(5))

C:\Users\nazih\AppData\Local\Temp\ipykernel_15908\1652544270.py:6: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
C:\Users\nazih\AppData\Local\Temp\ipykernel_15908\1652544270.py:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')


benign dataset shape: (297063, 136)
   stream               src_mac            dst_mac           src_ip  \
0       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   
1       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   
2       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   
3       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   
4       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   

         dst_ip  src_port  dst_port  inter_arrival_time  \
0  99.81.244.93     56891       443            0.000000   
1  99.81.244.93     56891       443            0.000164   
2  99.81.244.93     56891       443            0.000269   
3  99.81.244.93     56891       443            0.000414   
4  99.81.244.93     56891       443            0.001800   

   time_since_previously_displayed_frame  port_class_dst  ...   min_p   max_p  \
0                               0.000000               1  ...  2908.0  2908.0   
1                   

In [158]:
import pandas as pd

# Load and combine datasets (as before)
datasets = {
    'benign': pd.read_csv('labeled_dataset_benign.csv'),
    'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
    'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')
}
combined_df = pd.concat([datasets['benign'], datasets['dns'], datasets['mitm']], ignore_index=True)
combined_df = combined_df.drop('dst_mac', axis=1)

print(combined_df.shape)


C:\Users\nazih\AppData\Local\Temp\ipykernel_15908\2679432569.py:6: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
C:\Users\nazih\AppData\Local\Temp\ipykernel_15908\2679432569.py:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')


(883525, 135)


In [159]:
import re

def is_hex_mac(mac):
    if pd.isna(mac) or mac == 'unknown':  # Skip NaN or filled values
        return False
    pattern = r'^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$'
    return bool(re.match(pattern, str(mac)))

In [160]:
# Remove rows where src_mac is a hex MAC address
combined_df = combined_df[~combined_df['src_mac'].apply(is_hex_mac)]

In [161]:
print(combined_df.isnull().sum())
print(f"New shape before dropping missing values: {combined_df.shape}")

df = combined_df.dropna()
print(f"New shape after dropping missing values: {df.shape}")
print(f"Remaining missing values: {df.isnull().sum().sum()}")


stream                 0
src_mac                0
src_ip             20034
dst_ip             20034
src_port               0
                   ...  
q3_p                   0
q1_p                   0
iqr_p                  0
l3_ip_dst_count        0
Label                  0
Length: 135, dtype: int64
New shape before dropping missing values: (432555, 135)
New shape after dropping missing values: (307, 135)
Remaining missing values: 0


In [162]:
print(f"Number of duplicate rows: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"New shape after removing duplicates: {df.shape}")

Number of duplicate rows: 0
New shape after removing duplicates: (307, 135)


In [163]:
# Features (X) and targets
X = df.drop(['Label', 'src_mac'], axis=1, errors='ignore')  
y_attack = df['Label']  # For attack detection
y_device = df['src_mac']  # For device identification



In [164]:
print(y_device.unique())

['Teckin Plug 2' 'AeoTec Smart Home Hub' 'Home Eye Camera'
 'harman kardon (Ampak Technology)' 'SmartThings Hub'
 'Google Nest Mini Speaker' 'Borun/Sichuan-AI Camera' 'Philips Hue Bridge'
 'Eufy HomeBase 2' 'Amazon Echo Studio' 'Amazon Echo Show'
 'TP-Link Tapo Camera' 'Teckin Plug 1' 'Yi Indoor 2 Camera' 'Wyze Camera'
 'Amazon Echo Spot' 'Yi Indoor Camera' 'Amazon Echo Dot 1'
 'Arlo Base Station' 'Levoit Air Purifier' 'Yi Outdoor Camera'
 'Rbcior Camera']


In [165]:
print(y_attack.value_counts())

Label
DNS_Spoofing        303
BenignTraffic         3
MITM-ArpSpoofing      1
Name: count, dtype: int64


In [166]:


print(X.dtypes)

stream               int64
src_ip              object
dst_ip              object
src_port             int64
dst_port             int64
                    ...   
var_p              float64
q3_p               float64
q1_p               float64
iqr_p              float64
l3_ip_dst_count    float64
Length: 133, dtype: object


In [167]:
categorical_cols = [col for col in X.columns if X[col].nunique() < 10 or X[col].dtype == 'object']
print(f"Categorical features: {categorical_cols}")

Categorical features: ['src_ip', 'dst_ip', 'dst_port', 'port_class_dst', 'l4_tcp', 'l4_udp', 'ttl', 'tcp_window_size', 'handshake_version', 'handshake_cipher_suites_length', 'handshake_ciphersuites', 'handshake_extensions_length', 'tls_server', 'handshake_sig_hash_alg_len', 'http_request_method', 'http_host', 'http_response_code', 'user_agent', 'dns_server', 'dns_query_type', 'dns_len_ans', 'device_mac', 'eth_src_oui', 'eth_dst_oui', 'highest_layer', 'http_uri', 'http_content_len', 'http_content_type', 'icmp_type', 'icmp_checksum_status', 'icmp_data_size', 'stream_1_count', 'stream_5_count', 'stream_10_count', 'stream_30_count', 'ntp_interval']


In [168]:
for col in categorical_cols:
    X[col] = X[col].astype(str).fillna('unknown')

#Fill numerical columns with median (or mean)
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())



In [169]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_cols = [col for col in X.columns if X[col].nunique() < 10 or X[col].dtype == 'object']
le = LabelEncoder()

for col in categorical_cols:
    X[col] = le.fit_transform(X[col].astype(str).fillna('unknown'))



In [170]:
print(X.dtypes)

stream               int64
src_ip               int32
dst_ip               int32
src_port             int64
dst_port             int32
                    ...   
var_p              float64
q3_p               float64
q1_p               float64
iqr_p              float64
l3_ip_dst_count    float64
Length: 133, dtype: object


In [171]:


le_device = LabelEncoder()
y_device_encoded = le_device.fit_transform(y_device)
print("Encoding completed successfully.")
print(f"Updated X columns: {X.columns.tolist()}")

Encoding completed successfully.
Updated X columns: ['stream', 'src_ip', 'dst_ip', 'src_port', 'dst_port', 'inter_arrival_time', 'time_since_previously_displayed_frame', 'port_class_dst', 'l4_tcp', 'l4_udp', 'ttl', 'eth_size', 'tcp_window_size', 'payload_entropy', 'handshake_version', 'handshake_cipher_suites_length', 'handshake_ciphersuites', 'handshake_extensions_length', 'tls_server', 'handshake_sig_hash_alg_len', 'http_request_method', 'http_host', 'http_response_code', 'user_agent', 'dns_server', 'dns_query_type', 'dns_len_qry', 'dns_interval', 'dns_len_ans', 'device_mac', 'eth_src_oui', 'eth_dst_oui', 'payload_length', 'highest_layer', 'http_uri', 'http_content_len', 'http_content_type', 'icmp_type', 'icmp_checksum_status', 'icmp_data_size', 'jitter', 'stream_1_count', 'stream_1_mean', 'stream_1_var', 'src_ip_1_count', 'src_ip_1_mean', 'src_ip_1_var', 'src_ip_mac_1_count', 'src_ip_mac_1_mean', 'src_ip_mac_1_var', 'channel_1_count', 'channel_1_mean', 'channel_1_var', 'stream_jitte

In [172]:
from sklearn.model_selection import train_test_split

# Split for attack detection
X_train_attack, X_test_attack, y_train_attack, y_test_attack = train_test_split(
    X, y_attack, test_size=0.2, random_state=42, stratify=y_attack
)

# Split for device identification
X_train_device, X_test_device, y_train_device, y_test_device = train_test_split(
    X, y_device_encoded, test_size=0.2, random_state=42, stratify=y_device_encoded
)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [ ]:
# Train attack detection model
rf_model_attack = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
rf_model_attack.fit(X_train_attack, y_train_attack)

# Train device identification model
rf_model_device = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
rf_model_device.fit(X_train_device, y_train_device)

KeyboardInterrupt: 

In [ ]:
# Train attack detection model
xgb_model_attack = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_model_attack.fit(X_train_attack, y_train_attack)

# Train device identification model
xgb_model_device = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_model_device.fit(X_train_device, y_train_device)

In [ ]:
# Evaluate Random Forest for attack detection
rf_y_pred_attack = rf_model_attack.predict(X_test_attack)
print("Random Forest - Attack Detection Performance:")
print(f"Accuracy: {accuracy_score(y_test_attack, rf_y_pred_attack):.2f}")
print(classification_report(y_test_attack, rf_y_pred_attack))

# Evaluate XGBoost for attack detection
xgb_y_pred_attack = xgb_model_attack.predict(X_test_attack)
print("\nXGBoost - Attack Detection Performance:")
print(f"Accuracy: {accuracy_score(y_test_attack, xgb_y_pred_attack):.2f}")
print(classification_report(y_test_attack, xgb_y_pred_attack))

# Evaluate Random Forest for device identification
rf_y_pred_device = rf_model_device.predict(X_test_device)
print("\nRandom Forest - Device Identification Performance:")
print(f"Accuracy: {accuracy_score(y_test_device, rf_y_pred_device):.2f}")
print(classification_report(y_test_device, rf_y_pred_device, target_names=le_device.classes_))

# Evaluate XGBoost for device identification
xgb_y_pred_device = xgb_model_device.predict(X_test_device)
print("\nXGBoost - Device Identification Performance:")
print(f"Accuracy: {accuracy_score(y_test_device, xgb_y_pred_device):.2f}")
print(classification_report(y_test_device, xgb_y_pred_device, target_names=le_device.classes_))